In [1]:
##augmentation bit

In [2]:
import pandas as pd
import torch
data = pd.read_csv('data/public_data_dev/track_a/train/eng.csv')
# device = torch.device('cpu')
# print(torch.device) 

In [ ]:
#Multi-label backtranslation only


from tqdm import tqdm
import torch
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load translation models
en_to_fr_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr").to(device)
en_to_fr_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

fr_to_en_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-fr-en").to(device)
fr_to_en_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en")

# Back-translation function
def back_translate(text, en_to_x_model, en_to_x_tokenizer, x_to_en_model, x_to_en_tokenizer, device):
    try:
        # Translate from English to target language
        en_to_x = en_to_x_tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
        x_output = en_to_x_model.generate(**en_to_x)
        x_text = en_to_x_tokenizer.decode(x_output[0], skip_special_tokens=True)

        # Translate back from target language to English
        x_to_en = x_to_en_tokenizer(x_text, return_tensors="pt", truncation=True, max_length=512).to(device)
        en_output = x_to_en_model.generate(**x_to_en)
        back_translated_text = x_to_en_tokenizer.decode(en_output[0], skip_special_tokens=True)

        return back_translated_text
    except Exception as e:
        print(f"Error during back-translation: {e}")
        return text  # Return original text if an error occurs

# Augmentation function for multi-label data
def augment_minority_classes(data, classes, target_size, device):
    # Find samples that belong to any minority class
    minority_samples = data[data[classes].sum(axis=1) > 0]
    augmented_texts = []

    # Calculate the number of samples needed for each class
    class_sizes = data[classes].sum()
    samples_needed = {cls: max(0, target_size - class_sizes[cls]) for cls in classes}

    # Progress bar for augmentation
    for cls in classes:
        if samples_needed[cls] <= 0:
            continue  # Skip if no augmentation is needed

        class_texts = minority_samples[minority_samples[cls] == 1]
        for i in tqdm(range(samples_needed[cls]), desc=f"Augmenting {cls}", unit="sample"):
            text = class_texts.iloc[i % len(class_texts)]['text']
            labels = class_texts.iloc[i % len(class_texts)][classes].values

            # Back-translate the text
            augmented_text = back_translate(
                text, en_to_fr_model, en_to_fr_tokenizer, fr_to_en_model, fr_to_en_tokenizer, device
            )

            # Append augmented text with original labels
            augmented_texts.append({'text': augmented_text, **dict(zip(classes, labels))})

    return pd.DataFrame(augmented_texts)

# Loading  dataset
data = pd.read_csv("data/public_data_dev/track_a/train/eng.csv") 

# classes to balance
classes = ['anger', 'fear', 'joy', 'sadness', 'surprise']

# Handle missing values
data[classes] = data[classes].fillna(0)

# Calculate target size (size of the largest class)
largest_class_size = data[classes].sum().max()

# Augment minority classes
augmented_data = augment_minority_classes(data, classes, largest_class_size, device)

# Merge original and augmented data
balanced_data = pd.concat([data, augmented_data], ignore_index=True)

# Handle missing values in augmented dataset
balanced_data[classes] = balanced_data[classes].fillna(0)

# Print original and balanced dataset sizes
print(f"Original dataset size: {len(data)}")
print(f"Balanced dataset size: {len(balanced_data)}")

# Save the balanced dataset to a CSV file
output_file_path = '/home/h5/hama461f/Project/backtranslated.csv'
balanced_data.to_csv(output_file_path, index=False)

# Confirm the path where the file saved
print(f"Balanced dataset saved to {output_file_path}")

In [7]:
#multi label paraphrapse and back translate

import random
import torch
import pandas as pd
from tqdm import tqdm
from transformers import MarianMTModel, MarianTokenizer, PegasusForConditionalGeneration, PegasusTokenizer


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load translation models for multiple language pairs and store them in a dictionary
def load_translation_models(lang_pair, device):
    en_to_x_model = MarianMTModel.from_pretrained(f"Helsinki-NLP/opus-mt-{lang_pair[0]}-{lang_pair[1]}").to(device)
    en_to_x_tokenizer = MarianTokenizer.from_pretrained(f"Helsinki-NLP/opus-mt-{lang_pair[0]}-{lang_pair[1]}")
    x_to_en_model = MarianMTModel.from_pretrained(f"Helsinki-NLP/opus-mt-{lang_pair[1]}-{lang_pair[0]}").to(device)
    x_to_en_tokenizer = MarianTokenizer.from_pretrained(f"Helsinki-NLP/opus-mt-{lang_pair[1]}-{lang_pair[0]}")
    return en_to_x_model, en_to_x_tokenizer, x_to_en_model, x_to_en_tokenizer

# Preload all translation models for different language pairs
language_pairs = [
    ('en', 'fr'),  # English to French
    ('en', 'de'),  # English to German
    ('en', 'es'),  # English to Spanish
    ('en', 'it')   # English to Italian
]

translation_models = {}

# Load models for each language pair and store them in a dictionary
for lang_pair in language_pairs:
    en_to_x_model, en_to_x_tokenizer, x_to_en_model, x_to_en_tokenizer = load_translation_models(lang_pair, device)
    translation_models[lang_pair] = {
        'en_to_x_model': en_to_x_model,
        'en_to_x_tokenizer': en_to_x_tokenizer,
        'x_to_en_model': x_to_en_model,
        'x_to_en_tokenizer': x_to_en_tokenizer
    }

# Load paraphrasing model
paraphrase_model_name = "tuner007/pegasus_paraphrase"
paraphrase_model = PegasusForConditionalGeneration.from_pretrained(paraphrase_model_name).to(device)
paraphrase_tokenizer = PegasusTokenizer.from_pretrained(paraphrase_model_name)

# Define back-translation function with error handling
def back_translate(text, en_to_x_model, en_to_x_tokenizer, x_to_en_model, x_to_en_tokenizer, device):
    try:
        # Translate from English to target language
        en_to_x = en_to_x_tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
        x_output = en_to_x_model.generate(**en_to_x)
        x_text = en_to_x_tokenizer.decode(x_output[0], skip_special_tokens=True)

        # Translate back from target language to English
        x_to_en = x_to_en_tokenizer(x_text, return_tensors="pt", truncation=True, max_length=512).to(device)
        en_output = x_to_en_model.generate(**x_to_en)
        back_translated_text = x_to_en_tokenizer.decode(en_output[0], skip_special_tokens=True)

        return back_translated_text
    except Exception as e:
        print(f"Error during back-translation: {e}")
        return text  # Return original text if an error occurs

# Paraphrasing function using Pegasus with error handling
def paraphrase_text_pegasus(text):
    try:
        inputs = paraphrase_tokenizer([text], max_length=60, truncation=True, return_tensors="pt").to(device)
        paraphrase = paraphrase_model.generate(inputs["input_ids"], max_length=60, num_beams=5, num_return_sequences=1, early_stopping=True).to(device)
        paraphrased_text = paraphrase_tokenizer.decode(paraphrase[0], skip_special_tokens=True)
        return paraphrased_text
    except Exception as e:
        print(f"Error during paraphrasing: {e}")
        return text  # Return original text if an error occurs

# Augment text using back-translation and paraphrasing
def augment_text(text, language_pairs, device):
    # Back-translate using a fixed language pair
    lang_pair = random.choice(language_pairs)
    models = translation_models[lang_pair]
    augmented_text = back_translate(text, models['en_to_x_model'], models['en_to_x_tokenizer'], models['x_to_en_model'], models['x_to_en_tokenizer'], device)
    
    # Paraphrase the back-translated text
    paraphrased_text = paraphrase_text_pegasus(augmented_text)
    
    return paraphrased_text

# Augmentation function for minority classes with controlled augmentation factor
def augment_minority_classes_with_variability(data, classes, augmentation_factors, device, language_pairs):
    augmented_dfs = []

    for cls, factor in augmentation_factors.items():
        class_texts = data[data[cls] == 1]
        original_size = len(class_texts)
        samples_needed = int(original_size * factor) - original_size

        if samples_needed <= 0:
            continue  # Skip if no augmentation is needed

        augmented_texts = []

        for i in tqdm(range(samples_needed), desc=f"Augmenting {cls} (x{factor})", unit="sample"):
            text = class_texts.iloc[i % original_size]['text']
            labels = class_texts.iloc[i % original_size][classes].values
            
            # Augment text
            augmented_text = augment_text(text, language_pairs, device)
            
            # Append augmented text with original labels
            augmented_texts.append({'text': augmented_text, **dict(zip(classes, labels))})

        augmented_dfs.append(pd.DataFrame(augmented_texts))

    return pd.concat(augmented_dfs, ignore_index=True)

# dataset
data = pd.read_csv("data/public_data_dev/track_a/train/eng.csv") 

# Define augmentation factors for each class
augmentation_factors = {
    'anger': 3,  # Augment anger class 3x
    'fear': 0,   
    'joy': 2,    # Augment joy class 2x
    'sadness': 2, # augment sadness class 2x
    'surprise': 2 # Augment surprise class 2x
}

# Augment minority classes
augmented_data = augment_minority_classes_with_variability(data, ['anger', 'fear', 'joy', 'sadness', 'surprise'], augmentation_factors, device, language_pairs)

# Combine original and augmented data
balanced_data = pd.concat([data, augmented_data], ignore_index=True)

# Handle missing values in the augmented dataset
balanced_data[['anger', 'fear', 'joy', 'sadness', 'surprise']] = balanced_data[['anger', 'fear', 'joy', 'sadness', 'surprise']].fillna(0)

# Print original and balanced dataset sizes
print(f"Original dataset size: {len(data)}")
print(f"Balanced dataset size: {len(balanced_data)}")

# Save the balanced dataset to a CSV file
output_file_path = '/home/h5/hama461f/Project/backtrans_paraph_3222.csv'
balanced_data.to_csv(output_file_path, index=False)

# Confirm the path where the file is saved
print(f"Balanced dataset saved to {output_file_path}")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Augmenting surprise (x2): 100%|██████████| 839/839 [05:45<00:00,  2.43sample/s]


Original dataset size: 2768
Balanced dataset size: 5825
Balanced dataset saved to /home/h5/hama461f/Project/2attbalanced_data_paraphrase_fixed.csv


In [2]:
#2nd attempt multi label paraphrapse and back translate 5222

import random
import torch
import pandas as pd
from tqdm import tqdm
from transformers import MarianMTModel, MarianTokenizer, PegasusForConditionalGeneration, PegasusTokenizer


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load translation models for multiple language pairs and store them in a dictionary
def load_translation_models(lang_pair, device):
    en_to_x_model = MarianMTModel.from_pretrained(f"Helsinki-NLP/opus-mt-{lang_pair[0]}-{lang_pair[1]}").to(device)
    en_to_x_tokenizer = MarianTokenizer.from_pretrained(f"Helsinki-NLP/opus-mt-{lang_pair[0]}-{lang_pair[1]}")
    x_to_en_model = MarianMTModel.from_pretrained(f"Helsinki-NLP/opus-mt-{lang_pair[1]}-{lang_pair[0]}").to(device)
    x_to_en_tokenizer = MarianTokenizer.from_pretrained(f"Helsinki-NLP/opus-mt-{lang_pair[1]}-{lang_pair[0]}")
    return en_to_x_model, en_to_x_tokenizer, x_to_en_model, x_to_en_tokenizer

# Preload all translation models for different language pairs
language_pairs = [
    ('en', 'fr'),  # English to French
    ('en', 'de'),  # English to German
    ('en', 'es'),  # English to Spanish
    ('en', 'it')   # English to Italian
]

translation_models = {}

# Load models for each language pair and store them in a dictionary
for lang_pair in language_pairs:
    en_to_x_model, en_to_x_tokenizer, x_to_en_model, x_to_en_tokenizer = load_translation_models(lang_pair, device)
    translation_models[lang_pair] = {
        'en_to_x_model': en_to_x_model,
        'en_to_x_tokenizer': en_to_x_tokenizer,
        'x_to_en_model': x_to_en_model,
        'x_to_en_tokenizer': x_to_en_tokenizer
    }

# Load paraphrasing model
paraphrase_model_name = "tuner007/pegasus_paraphrase"
paraphrase_model = PegasusForConditionalGeneration.from_pretrained(paraphrase_model_name).to(device)
paraphrase_tokenizer = PegasusTokenizer.from_pretrained(paraphrase_model_name)

# Define back-translation function with error handling
def back_translate(text, en_to_x_model, en_to_x_tokenizer, x_to_en_model, x_to_en_tokenizer, device):
    try:
        # Translate from English to target language
        en_to_x = en_to_x_tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
        x_output = en_to_x_model.generate(**en_to_x)
        x_text = en_to_x_tokenizer.decode(x_output[0], skip_special_tokens=True)

        # Translate back from target language to English
        x_to_en = x_to_en_tokenizer(x_text, return_tensors="pt", truncation=True, max_length=512).to(device)
        en_output = x_to_en_model.generate(**x_to_en)
        back_translated_text = x_to_en_tokenizer.decode(en_output[0], skip_special_tokens=True)

        return back_translated_text
    except Exception as e:
        print(f"Error during back-translation: {e}")
        return text  # Return original text if an error occurs

# Paraphrasing function using Pegasus with error handling
def paraphrase_text_pegasus(text):
    try:
        inputs = paraphrase_tokenizer([text], max_length=60, truncation=True, return_tensors="pt").to(device)
        paraphrase = paraphrase_model.generate(inputs["input_ids"], max_length=60, num_beams=5, num_return_sequences=1, early_stopping=True).to(device)
        paraphrased_text = paraphrase_tokenizer.decode(paraphrase[0], skip_special_tokens=True)
        return paraphrased_text
    except Exception as e:
        print(f"Error during paraphrasing: {e}")
        return text  # Return original text if an error occurs

# Augment text using back-translation and paraphrasing
def augment_text(text, language_pairs, device):
    # Back-translate using a fixed language pair
    lang_pair = random.choice(language_pairs)
    models = translation_models[lang_pair]
    augmented_text = back_translate(text, models['en_to_x_model'], models['en_to_x_tokenizer'], models['x_to_en_model'], models['x_to_en_tokenizer'], device)
    
    # Paraphrase the back-translated text
    paraphrased_text = paraphrase_text_pegasus(augmented_text)
    
    return paraphrased_text

# Augmentation function for minority classes with controlled augmentation factor
def augment_minority_classes_with_variability(data, classes, augmentation_factors, device, language_pairs):
    augmented_dfs = []

    for cls, factor in augmentation_factors.items():
        class_texts = data[data[cls] == 1]
        original_size = len(class_texts)
        samples_needed = int(original_size * factor) - original_size

        if samples_needed <= 0:
            continue  # Skip if no augmentation is needed

        augmented_texts = []

        for i in tqdm(range(samples_needed), desc=f"Augmenting {cls} (x{factor})", unit="sample"):
            text = class_texts.iloc[i % original_size]['text']
            labels = class_texts.iloc[i % original_size][classes].values
            
            # Augment text
            augmented_text = augment_text(text, language_pairs, device)
            
            # Append augmented text with original labels
            augmented_texts.append({'text': augmented_text, **dict(zip(classes, labels))})

        augmented_dfs.append(pd.DataFrame(augmented_texts))

    return pd.concat(augmented_dfs, ignore_index=True)

# dataset
data = pd.read_csv("data/public_data_dev/track_a/train/eng.csv")  # Replace with your dataset path

# Define augmentation factors for each class
augmentation_factors = {
    'anger': 5,  # Augment anger class 5x
    'fear': 0,   
    'joy': 2,    # Augment joy class 2x
    'sadness': 2, # Augment sadness class 2x
    'surprise': 2 # Augment surprise class 2x
}

# Augment minority classes
augmented_data = augment_minority_classes_with_variability(data, ['anger', 'fear', 'joy', 'sadness', 'surprise'], augmentation_factors, device, language_pairs)

# Combine original and augmented data
balanced_data = pd.concat([data, augmented_data], ignore_index=True)

# Handle missing values in the augmented dataset
balanced_data[['anger', 'fear', 'joy', 'sadness', 'surprise']] = balanced_data[['anger', 'fear', 'joy', 'sadness', 'surprise']].fillna(0)

# Print original and balanced dataset sizes
print(f"Original dataset size: {len(data)}")
print(f"Balanced dataset size: {len(balanced_data)}")

# Print class distribution in balanced dataset
class_counts = balanced_data[['anger', 'fear', 'joy', 'sadness', 'surprise']].sum()
print("\nClass distribution in balanced dataset:")
print(class_counts)

# Save the balanced dataset to a CSV file
output_file_path = '/home/h5/hama461f/Project/backtrans_paraph_3222.csv'
balanced_data.to_csv(output_file_path, index=False)

# Confirm the path where the file is saved
print(f"Balanced dataset saved to {output_file_path}")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Augmenting surprise (x2): 100%|██████████| 839/839 [05:49<00:00,  2.40sample/s]

Original dataset size: 2768
Balanced dataset size: 6491

Class distribution in balanced dataset:
anger       1938
fear        3938
joy         1580
sadness     2619
surprise    2483
dtype: int64
Balanced dataset saved to /home/h5/hama461f/Project/3attequalbalanced_data_paraphrase_fixed.csv
